In [26]:
import kaggle
import pandas as pd
import numpy as np
import zipfile
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import metrics 
from sklearn.neural_network import MLPClassifier

from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import matplotlib.pyplot as plt
width = 20
height = 8
sns.set(rc = {'figure.figsize':(width, height)})

In [27]:
df_train = pd.read_csv('df_clean.csv')

In [28]:
exog = ['First_choice', 'Single', 'Application_mode_1st_phase', 'Application_mode_2nd_phase', 'Application_mode_Over_23 years_old',
        'Application_mode_Tech_Spec', 'Application_mode_Change', 'Quali_Secondary education', 'Quali_Basic education', 
        'Quali_Tech Spec course', 'Quali_Higher education', 'Quali_Other', 'Quali_12th year of schooling', 'Quali_Higher education',
        'Quali_Professional higher technical course', 'M_Quali_Secondary education', 'M_Quali_Basic education_3rd', 'M_Quali_Basic education_1st',
        'M_Quali_Basic education_2nd', 'M_Quali_Higher Education', 'M_Quali_Secondary education', 'M_Quali_Basic education_3rd',
        'M_Quali_Basic education_1st', 'M_Quali_Basic education_2nd', 'M_Quali_Higher Education', 'F_Quali_Secondary education',
        'F_Quali_Basic education_3rd', 'F_Quali_Basic education_1st', 'F_Quali_Basic education_2nd', 'F_Quali_Higher Education',
        'Daytime/evening attendance', 'Previous qualification (grade)', 'Admission grade', 'Debtor', 'Tuition fees up to date',
        'Gender', 'Scholarship holder', 'Age at enrollment', 'Curricular units 2nd sem (approved)', 'Curricular units 1st sem (approved)',
        'Curricular units 2nd sem (grade)', 'Curricular units 1st sem (grade)']

In [29]:
X_train = df_train[exog]
X_train = sm.add_constant(X_train)

Y_train_g = df_train['Target_Graduate']
Y_train_e = df_train['Target_Enrolled']
Y_train_d = df_train['Target_Dropout']

X_train = X_train.astype(float)

Y_train_g = Y_train_g.astype(float)
Y_train_e = Y_train_e.astype(float)
Y_train_d = Y_train_d.astype(float)

model_g = sm.Logit(Y_train_g, X_train, missing='drop').fit_regularized()
model_e = sm.Logit(Y_train_e, X_train, missing='drop').fit_regularized()
model_d = sm.Logit(Y_train_d, X_train, missing='drop').fit_regularized()

train_pred = pd.DataFrame()
train_pred['Graduate'] = model_g.predict(X_train)
train_pred['Enrolled'] = model_e.predict(X_train)
train_pred['Dropout'] = model_d.predict(X_train)

# print(model.summary())
# np.round(np.mean((model.predict(X_train)>0.5) == df['Target_Enrolled']),4)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3335199319537609
            Iterations: 268
            Function evaluations: 278
            Gradient evaluations: 268


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4056484673146826
            Iterations: 259
            Function evaluations: 268
            Gradient evaluations: 259
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.25806733177346447
            Iterations: 264
            Function evaluations: 273
            Gradient evaluations: 264


In [38]:
def classify(x):
    if x['Graduate'] > x['Enrolled'] and x['Graduate'] > x['Dropout']:
        return 'Graduate'
    if x['Enrolled'] > x['Graduate'] and x['Enrolled'] > x['Dropout']:
        return 'Enrolled'
    if x['Dropout'] > x['Enrolled'] and x['Dropout'] > x['Graduate']:
        return 'Dropout'
    else:
        return 'Graduate'
    

def target_to_num(x):
    if x['Target'] == 'Graduate':
        return 1
    if x['Target'] == 'Dropout':
        return -1
    if x['Target'] == 'Enrolled':
        return 0
    
def num_to_target(x):
    if x['Target_num'] == 1:
        return 'Graduate'
    if x['Target_num'] == -1:
        return 'Dropout'
    if x['Target_num'] == 0:
        return 'Enrolled'
    
df_train['Target_num'] = df_train.apply(lambda x: target_to_num(x), axis=1)

In [31]:
train_pred['Target'] = train_pred.apply(lambda x: classify(x), axis=1)
print(np.round(np.mean(train_pred['Target'] == df_train['Target']),4))
print(train_pred.head())

0.7946
   Graduate  Enrolled   Dropout    Target
0  0.913780  0.115706  0.015012  Graduate
1  0.013489  0.188903  0.853568   Dropout
2  0.019990  0.075185  0.896228   Dropout
3  0.963667  0.048383  0.013187  Graduate
4  0.742316  0.229608  0.043997  Graduate


In [32]:
df_test = pd.read_csv('df_clean_test.csv')
X_test = df_test[exog]
X_test = sm.add_constant(X_test)
X_test = X_test.astype(float)

test_pred = pd.DataFrame()
test_pred['Graduate'] = model_g.predict(X_test)
test_pred['Enrolled'] = model_e.predict(X_test)
test_pred['Dropout'] = model_d.predict(X_test)

test_pred['Target'] = test_pred.apply(lambda x: classify(x), axis=1)

submission = pd.DataFrame()
submission['id'] = df_test['id']
submission['Target'] = test_pred['Target']

print(test_pred.head())
submission.to_csv('submission.csv', index=False)

   Graduate  Enrolled   Dropout    Target
0  0.002876  0.010527  0.982431   Dropout
1  0.752700  0.321279  0.031883  Graduate
2  0.802967  0.126724  0.033367  Graduate
3  0.707713  0.134234  0.053683  Graduate
4  0.125802  0.435977  0.265427  Enrolled


##### Score: 0.79543

##### Random Forest

In [45]:
X_train = df_train[exog]
Y_train_num = df_train['Target_num']

rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=0, n_jobs=-1)

param_grid = {"criterion":['gini', 'entropy', 'log_loss'], "min_samples_leaf" : [1, 2, 5], "min_samples_split" : [20], "n_estimators": [100]}

gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

gs = gs.fit(X_train, Y_train_num)

print(gs.best_score_)
print(gs.best_params_)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.815886458088293
{'criterion': 'gini', 'min_samples_leaf': 2, 'min_samples_split': 20, 'n_estimators': 100}


In [48]:
clf = RandomForestClassifier(random_state=7, n_estimators=100, min_samples_split=20, min_samples_leaf=2)
clf.fit(X_train, Y_train_num)

y_pred = clf.predict(X_train)

print("ACCURACY OF THE MODEL:", metrics.accuracy_score(Y_train_num, y_pred))

ACCURACY OF THE MODEL: 0.8564128701743381


In [49]:
df_test = pd.read_csv('df_clean_test.csv')
X_test = df_test[exog]

y_pred = clf.predict(X_test)

data = {'id':df_test['id'],
        'Target_num': y_pred.astype(int)}

submission = pd.DataFrame(data)
submission['Target'] = submission.apply(lambda x: num_to_target(x), axis=1)
submission.drop('Target_num', axis=1, inplace=True)

submission.to_csv('submission.csv', index=False)

##### Score: 0.81768